In [4]:
#other
import pandas as pd
import numpy as np
from tqdm import notebook
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score

#nlp
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
from pywsd.utils import lemmatize_sentence
import torch
import transformers as ppb
import re
from sklearn.feature_extraction.text import TfidfVectorizer

#model
from sklearn.linear_model import LogisticRegression


[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...


KeyboardInterrupt: 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
#path = "/content/drive/My Drive/toxic_comments.csv"
path = "/datasets/toxic_comments.csv"
tweets = pd.read_csv(path)

In [7]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Загрузили данные , и получили почти 160 тысяч твитов ,которые нужно предобработать и библиотеки нужны для английского языка

In [8]:
#regular expressions
tweets['text'][0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [9]:
re.sub(r"[^a-zA-Z]",' ', tweets['text'][0])

'Explanation Why the edits made under my username Hardcore Metallica Fan were reverted  They weren t vandalisms  just closure on some GAs after I voted at New York Dolls FAC  And please don t remove the template from the talk page since I m retired now             '

Пример обработки твитов с помощью регулярных выражений

In [10]:
corpus = list(tweets['text'].values.astype('U'))

In [5]:
def clear_text(text):
    text = re.sub(r"[^a-zA-Z']", ' ', text)
    text = text.split()
    return ' '.join(text)

In [6]:
def lemma(text):
    lemm_list = lemmatize_sentence(text)
    lemm_text = " ".join(lemm_list)
    return lemm_text

In [13]:
import resource, sys
resource.setrlimit(resource.RLIMIT_STACK, (resource.RLIM_INFINITY, resource.RLIM_INFINITY))
sys.setrecursionlimit(10**6)

Была проблема с ограничением на количество рекурсий ,поэтому пришлось подшаманить костылем из stack'а 

In [14]:
for i in notebook.tqdm(range(len(corpus))):
    corpus[i] = clear_text(corpus[i])
    corpus[i] = lemma(corpus[i])
    corpus[i] = corpus[i].replace("'m", "be").replace("'re","be").replace("'s", "be").replace("n't","not")

Провели лемматизацию а также очистку текста ,где также обработали случаю ,когда лемматизатор не может обработать апострафы - поэтому вручную их заменяем на лемматизированную форму

In [15]:
corpus[0]

'explanation why the edits make under my username hardcore metallica fan be revert they be not vandalisms just closure on some gas after i vote at new york doll fac and please do not remove the template from the talk page since i be retired now'

In [16]:
dataset = pd.DataFrame({'lemma': corpus, 'toxic': tweets['toxic']})

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
stopwords = set(nltk_stopwords.words('english'))

In [21]:
X_train, X_test , y_train, y_test = train_test_split(dataset['lemma'], dataset['toxic'], test_size = 0.3, random_state = 42)

In [22]:
count_tf_idf = TfidfVectorizer(stop_words = stopwords)

In [23]:
X_train

54568     conflict of interest note by your user name it...
3786      update actually i change this to something bla...
22938     mrca article sniperz thanks for your comment o...
137856    argue that bart and caltrain should get more m...
143038    blocked hi i block you for hour for be uncivil...
                                ...                        
119879    redirect talk john loveday experimental physicist
103694     back it up post the line here with the reference
131932    i wo not stop that sometimes germanic equal ge...
146867    british band i think you 've mistake scottish ...
121958    you be wrong justin thompson be mention in the...
Name: lemma, Length: 111699, dtype: object

In [24]:
tf_idf = count_tf_idf.fit_transform(X_train)

Перевели слова в векторный вид TF-IDF для последующей обработки их для моделей Логистической Регрессии и бустинга

<h1>Logistic Regression model</h1>

In [26]:
modelLR = LogisticRegression(solver = 'lbfgs', max_iter = 1000,class_weight='balanced')

In [27]:
mLR = modelLR.fit(tf_idf, y_train)

In [28]:
linear_score = cross_val_score(mLR, tf_idf, y_train, cv=5,scoring='f1')

In [29]:
linear_score.mean()

0.706969239553178

Просто линейной регрессии и TF-IDF не хватило для получения желаемой метрики , поэтому будет улучшать результаты дальше с помощью других моделей и методик

<h1>LightGBM model</h1>

In [30]:
from lightgbm import LGBMClassifier
import lightgbm as lgb

In [31]:
modelLight = LGBMClassifier(random_state = 42)
param_search = {'depth' : [-1],
                'learning_rate' : [0.1],
                'n_estimators' : [100,200,1000]
               }
gsearchLight = GridSearchCV(estimator=modelLight, cv=3,
                        param_grid=param_search,scoring = 'f1')
gsearchLight.fit(tf_idf, y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=100,
                                      n_jobs=-1, num_leaves=31, objective=None,
                                      random_state=42, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=True,
                                      subsample=1.0, subsample_for_bin=200000,
                                      subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid={'depth': [-1], 'learning_rate': [0.1],
                         'n_esti

In [32]:
print(gsearchLight.best_score_)
print(gsearchLight.best_params_)

0.7630467648169303
{'depth': -1, 'learning_rate': 0.1, 'n_estimators': 1000}


LightGBM справился с помощью подбора гиперпараметров и TF-IDF с желаемой метрикой. Для экономии времени ,ибо и так долго работает gridSearch , перебирали только возможные кол-ва итераций ( но было бы больше времени и не крашилось ,то еще бы точно перебрал learning_rate по-меньше )

<h1>distilBERT/BERT models</h1>

In [69]:
#distilBERT + LogReg
model_class, tokenizer_class, pretrained_weights = \
(ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

In [70]:
#load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [71]:
bert_set = dataset.sample(2000).reset_index(drop=True)

In [72]:
tokenized = bert_set['lemma'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)));

Token indices sequence length is longer than the specified maximum sequence length for this model (713 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (621 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (640 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (797 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [73]:
tokens = []
target = []
f = 0
for i in range(len(tokenized)):
    if len(tokenized[i]) < 400:
        tokens.append(tokenized[i])
        target.append(bert_set['toxic'][i])
tokens = (pd.Series(tokens)).head(1800)
target = (pd.Series(target)).head(1800)

In [74]:
max_len = 0
for i in tokens.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokens.values])

attention_mask = np.where(padded != 0, 1, 0)

In [75]:
batch_size = 200
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

Хотел делать вариант без батчей , а просто закинуть в модель 400+ самплов ,но почему-то в таком случае у меня просто умирал kernel от перегрузки на RAM. Я убирал цикл по padded.shape[0] , и просто передавал переменные без срезов ,то есть batch = torch.LongTensor(padded)  ,вместо среза на основе батча ( тоже самое делал и с маской )
<br>
Поэтому ранее взял head(400) в features и соотвественные таргеты к ним  ,немного кажется костыльным способом ,но иначе не знаю как сделать. Также очень интересно в случае если выборка не кратна рамзеру батчей ,то тогда отбрасываются оставшие с хвоста наблюдения или рандомно ? Просто на случай ,если рандомно ,то потом непонятно как сопоставлять features и target, как раз-таки на этот и случай сделал заранее отбор по head() кратный размеру батча

In [76]:
features = np.concatenate(embeddings)

In [80]:
modelDBertLR = LogisticRegression(solver = 'lbfgs', max_iter = 1000,class_weight='balanced')

In [82]:
mDBLR = modelDBertLR.fit(features, target)

In [83]:
disBert_score = cross_val_score(mDBLR, features, target, cv=5,scoring='f1')

In [84]:
disBert_score.mean()

0.6525678804748573

Прогнали также distilBert и получили вполне неплохие показатели для такой маленькой выборки

In [85]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

In [86]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [87]:
tokenized = bert_set['lemma'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)));

Token indices sequence length is longer than the specified maximum sequence length for this model (713 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (621 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (640 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (797 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (541 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [88]:
tokens = []
target = []
for i in range(len(tokenized)):
    if len(tokenized[i]) < 400:
        tokens.append(tokenized[i])
        target.append(bert_set['toxic'][i])
tokens = (pd.Series(tokens)).head(1800)
target = (pd.Series(target)).head(1800)

In [89]:
max_len = 0
for i in tokens.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokens.values])

attention_mask = np.where(padded != 0, 1, 0)

In [90]:
batch_size = 200
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

In [97]:
features = np.concatenate(embeddings)

In [105]:
modelBertLR = LogisticRegression(solver = 'lbfgs', max_iter = 1000,class_weight='balanced')

In [107]:
mBLR = modelBertLR.fit(features, target)

In [108]:
bert_score = cross_val_score(mBLR, features, target, cv=5,scoring='f1')

In [109]:
bert_score.mean()

0.6379581436703068

Тут я очень сильно удивился и понять не могу почему так , ибо Bert работает по сути лучше distilBert ,но дольше. Как оказалось второе правда и работает в 2 раза дольше ,но показатель метрики намного хуже ,что вводит меня в небольшой ступор почему может быть так 